In [20]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
from config import t_sat,flow_rate,project_span
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc,month_index

# Read files

In [21]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)

# Recovery Calculation

In [22]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()
# Create a dataframe with sequence as block_id, lift and lat_lng as columns
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
# Create block_list for later index reference.
block_list = block_leach_time.index.tolist()

In [23]:
# Set up project span
projectspan = (
    datetime.strptime(project_span['startdate'],'%Y-%m-%d'),
    datetime.strptime(project_span['enddate'],'%Y-%m-%d')
    )
# Create month list of the project
startyear = projectspan[0].year
startmonth = projectspan[0].month
endyear = projectspan[1].year
endmonth = projectspan[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Use monthlist create recovery dataframe
rec_df = pd.DataFrame(monthlist,columns = ['months'])

In [24]:
# This block of code calculates a specific block's recovery over months.
## k factor here will be changed to a formula
k = 0.001

## The function takes two block indices 
## and the k factor that is going to be used for recovery calculation.
def block_months_rec(i,ref,k,lift_diff):
    block = blocks_df.iloc[i]
    # if the two indices are the same, means this is calculating the primary leach recovery
    if i == ref:
        start_date = block['leach_start']
        end_date = block['leach_end']
    # if the two indices are different, means calculating recoveries of blocks under secondary leach
    else:
        # ref_block is the block on the very top
        ref_block = blocks_df.iloc[ref]
        # the blocks below the ref_block inherit the leaching dates from the ref_block
        # calculate leach delay
        start_date = ref_block['leach_start'] + timedelta(days = t_sat + round(lift_diff * flow_rate))
        end_date = ref_block['leach_end'] + timedelta(days = round(lift_diff * flow_rate))
    # locate the month row
    m_index = month_index(start_date,projectspan)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    rmax = block['rmax']
    # calcuate how much recovery that this block has achieved since the start
    rcum = sum(rec_df.iloc[0:m_index][i])
    if rcum < rmax: # if cumulative recovery has not achieved the maximum recovery
        for m in range(start_month,end_month+1):
            # calculate how many days in this month for recovery calculation
            if m == start_month:
                if m == 12:
                    days = 31 - start_date.day
                else:
                    days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
            elif m == end_month:
                days = end_date.day
            elif m < 12:
                days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
            elif m == 12:
                days = 31
            else:
                days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
            # recovery calculation here.
            r = recovery_calc(rmax,rcum,k,days)
            if rcum + r > rmax:
                r = rmax - rcum
                rcum = 1
            else:
                rcum += r
            # write the recovery to the rec_df.
            rec_df.at[m_index,i] = r
            m_index += 1

In [25]:
# Calculate recoveries for each block
for high in block_list:
    this_block = blocks_df.iloc[high]
    rec_df[high]=0.0
    lift_diff = 0
    block_months_rec(high,high,k,lift_diff)
    high_lift = this_block['lift']
    # After this block's recovery has been calculated,
    # start calculating any blocks that are below this block.
    for low in range(0,high):
        if lat_lng_list[high]==lat_lng_list[low]:
            low_lift = blocks_df.iloc[low]['lift']
            lift_diff = high_lift-low_lift
            block_months_rec(low,high,k,lift_diff)

In [26]:
# Create dataframe records ounces leached out from each block during months
oz_df = pd.DataFrame(rec_df.loc[0:,0:] * blocks_df.ounces_per_block)
oz_df.insert(0, 'months', monthlist)
oz_df['oz_monthly'] = oz_df.sum(axis = 1)
oz_df.head()

,months,0,1,2,3,4,5,6,7,8,...,5140,5141,5142,5143,5144,5145,5146,5147,5148,oz_monthly
0,2019-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,2019-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2019-03-01,0.016734,0.016734,0.016734,0.016734,0.016734,0.016734,0.016734,0.016734,0.016734,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.869524
3,2019-04-01,0.245461,0.245461,0.245461,0.245461,0.245461,0.245461,0.245461,0.245461,0.245461,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.118392
4,2019-05-01,0.243185,0.243185,0.243185,0.243185,0.243185,0.243185,0.243185,0.243185,0.243185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.034893


In [40]:
# Creating cells gold production stats by month.
blocks_oz_df = oz_df.set_index('months').T
## Export the data for visualization
file_path = os.path.join('..','..','Resources','blocks_oz.json')
blocks_oz_df.to_json(file_path)
blocks_oz_df.drop(blocks_oz_df.tail(1).index,inplace=True)
## Insert cell column for groupby.
blocks_oz_df.insert(0,'cell',blocks_df['cell'])
cell_oz_df = blocks_oz_df.groupby(['cell']).sum()
## Export the data for visualization
file_path = os.path.join('..','..','Resources','cells_oz.json')
cell_oz_df.to_json(file_path)

In [42]:
blocks_oz_df.insert(0,"lat",blocks_df['lat'])
blocks_oz_df.insert(1,"lng",blocks_df['lng'])
blocks_oz_df.head()

months,lat,lng,lat_lng,cell,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,...,2031-03-01 00:00:00,2031-04-01 00:00:00,2031-05-01 00:00:00,2031-06-01 00:00:00,2031-07-01 00:00:00,2031-08-01 00:00:00,2031-09-01 00:00:00,2031-10-01 00:00:00,2031-11-01 00:00:00,2031-12-01 00:00:00
0,65.012000,-147.351429,"[65.012, -147.35142857142858]",1,0.0,0.0,0.016734,0.245461,0.243185,0.225829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,65.012000,-147.350612,"[65.012, -147.35061224489797]",1,0.0,0.0,0.016734,0.245461,0.243185,0.225829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,65.012265,-147.353061,"[65.01226530612244, -147.3530612244898]",1,0.0,0.0,0.016734,0.245461,0.243185,0.225829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,65.012265,-147.352245,"[65.01226530612244, -147.3522448979592]",1,0.0,0.0,0.016734,0.245461,0.243185,0.225829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,65.012265,-147.351429,"[65.01226530612244, -147.35142857142858]",1,0.0,0.0,0.016734,0.245461,0.243185,0.225829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
blocks_oz_df = blocks_oz_df.groupby(['lat','lng']).sum()

In [44]:
blocks_oz_df

months                 cell  2019-01-01 00:00:00  2019-02-01 00:00:00  \
lat       lng                                                           
65.012000 -147.351429     1                  0.0                  0.0   
          -147.350612     1                  0.0                  0.0   
65.012265 -147.353061     1                  0.0                  0.0   
          -147.352245     1                  0.0                  0.0   
          -147.351429     1                  0.0                  0.0   
...                     ...                  ...                  ...   
65.024204 -147.369388     5                  0.0                  0.0   
          -147.368571     4                  0.0                  0.0   
          -147.367755     4                  0.0                  0.0   
          -147.366939     4                  0.0                  0.0   
          -147.366122     4                  0.0                  0.0   

months                 2019-03-01 00:00:00  2019-04-01 00:00:00  \
lat       lng                                                     
65.012000 -147.351429             0.016734             0.245461   
          -147.350612             0.016734             0.245461   
65.012265 -147.353061             0.016734             0.245461   
          -147.352245             0.016734             0.245461   
          -147.351429             0.016734             0.245461   
...                                    ...                  ...   
65.024204 -147.369388             0.000000             0.000000   
          -147.368571             0.000000             0.000000   
          -147.367755             0.000000             0.000000   
          -147.366939             0.000000             0.000000   
          -147.366122             0.000000             0.000000   

months                 2019-05-01 00:00:00  2019-06-01 00:00:00  \
lat       lng                                                     
65.012000 -147.351429             0.243185             0.225829   
          -147.350612             0.243185             0.225829   
65.012265 -147.353061             0.243185             0.225829   
          -147.352245             0.243185             0.225829   
          -147.351429             0.243185             0.225829   
...                                    ...                  ...   
65.024204 -147.369388             0.000000             0.000000   
          -147.368571             0.000000             0.017888   
          -147.367755             0.000000             0.017888   
          -147.366939             0.000000             0.017888   
          -147.366122             0.000000             0.017888   

months                 2019-07-01 00:00:00  2019-08-01 00:00:00  \
lat       lng                                                     
65.012000 -147.351429             0.202822             0.000000   
          -147.350612             0.202822             0.000000   
65.012265 -147.353061             0.202822             0.000000   
          -147.352245             0.202822             0.000000   
          -147.351429             0.202822             0.000000   
...                                    ...                  ...   
65.024204 -147.369388             0.010718             0.080986   
          -147.368571             0.180479             0.161933   
          -147.367755             0.180479             0.161933   
          -147.366939             0.180479             0.161933   
          -147.366122             0.180479             0.161933   

months                 2019-09-01 00:00:00  ...  2031-03-01 00:00:00  \
lat       lng                               ...                        
65.012000 -147.351429             0.000000  ...                  0.0   
          -147.350612             0.000000  ...                  0.0   
65.012265 -147.353061             0.000000  ...                  0.0   
          -147.352245             0.000000  ...                  0.0   
  

In [46]:
blocks_oz_df.reset_index().drop(columns = 'cell')

months,lat,lng,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,...,2031-03-01 00:00:00,2031-04-01 00:00:00,2031-05-01 00:00:00,2031-06-01 00:00:00,2031-07-01 00:00:00,2031-08-01 00:00:00,2031-09-01 00:00:00,2031-10-01 00:00:00,2031-11-01 00:00:00,2031-12-01 00:00:00
0,65.012000,-147.351429,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,65.012000,-147.350612,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,65.012265,-147.353061,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,65.012265,-147.352245,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,65.012265,-147.351429,0.0,0.0,0.016734,0.245461,0.243185,0.225829,0.202822,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,65.024204,-147.369388,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.010718,0.080986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1430,65.024204,-147.368571,0.0,0.0,0.000000,0.000000,0.000000,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1431,65.024204,-147.367755,0.0,0.0,0.000000,0.000000,0.000000,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1432,65.024204,-147.366939,0.0,0.0,0.000000,0.000000,0.000000,0.017888,0.180479,0.161933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# create dataframe for each blocks' remaining ounces over months.
#remain_oz_df

In [29]:
# Creating blocks cumulative gold production
#cum_oz_df=oz_df[]
#cum_oz_df.head()

In [30]:
#production = f(k,dataframe)
#f(production, actual_production) = k